# Assignment #2

Abhilash Vikram Gupta, Hao Lun Chu Colin, Matthew Viteri, Stephen Oluwaniyi and Yeggi Lee

In [1]:
import numpy as np
import pandas as pd
import tweepy
import GetOldTweets3 as got

from configparser import ConfigParser
from IPython.display import display

### A. Collect 5k tweets on the 2020 Presidential election. It is necessary to use general hashtags and search keywords (like 2020 U.S. election or 2020 Presidential Election) rather than “Trump” or “Biden” or “Warren”. Use Tweepy in your Python script, and also ask for the location specified by the user during registration with Twitter. 

In [2]:
def get_creds():
    config = ConfigParser()
    config.read('./credentials.ini')

    consumer_section = config['consumer']
    api_key = consumer_section['api-key']
    api_secret_key = consumer_section['api-secret-key']

    access_section = config['access']
    access_token = access_section['access-token']
    access_token_secret = access_section['access-token-secret']
    
    return api_key, api_secret_key, access_token, access_token_secret

In [43]:
try:
    consumer_key, consumer_secret, access_token, access_token_secret = get_creds()
except KeyError as e:
    raise IOError('Twitter credentials used are not included as part of the submitted assignment.') from e

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
display('Preparing...')
df = pd.DataFrame(columns=['Date', 'Username', 'User Location', 'Tweet'])

query = 'president election 2020'
start = '2018-01-01'
end = '2019-09-01'
count = 5000
tweetCriteria = (got.manager.TweetCriteria().setQuerySearch(query)
                 .setSince(start)
                 .setUntil(end)
                 .setMaxTweets(count))

tweets = got.manager.TweetManager.getTweets(tweetCriteria)

display('Collecting tweets...')
for tweet in tweets:
    df = df.append({
        'Date': tweet.date,
        'Username': tweet.username,
        'Tweet': tweet.text
    }, ignore_index=True)

display('Collecting user locations...')
for _, row in df.iterrows():
    if np.isnan(row['User Location']):
        row['User Location'] = api.get_user(row['Username']).location

display('Done.')
display(df)
df.to_csv(f'./{query}_({count})_{end}.csv', index=False)

In [41]:
query = 'president election 2020'
start = '2018-01-01'
end = '2019-09-01'
count = 5000
df.to_csv(f'./{query}_({count})_{end}.csv', index=False)

In [42]:
df = pd.read_csv('president_election_2020_2019-08-31.csv', dtype={'User Location': str})

In [ ]:
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

for index, row in df.iterrows():
    if pd.isna(row['User Location']):
        try:
            df.at[index, 'User Location'] = api.get_user(row['Username']).location
        except:
            df.to_csv(f'./{query}_({count})_{end}.csv', index=False)
            display(df[df['User Location'].isnull()])
            break

Rate limit reached. Sleeping for: 264


### B.   Find FOUR key issues mentioned by the public in the tweets – e.g., character, personality, healthcare, crime, jobs, etc. (these are examples only, use wordfrequency analysis to find out). Replace words if necessary. Issues associated with Mr. Trump may not have a strong association with the democratic candidate and vice versa. So find a couple of issues that are likely to have strong association with Mr. Trump, and a couple for the democratic candidate.

In [ ]:
df = pd.read_csv('president_election_2020_2019-08-31.csv', index_col=0)